In [1]:
import pickle
import re

In [3]:
with open("all_preproc_no_hashtags_really.pickle", 'rb') as f:
    data = pickle.load(f)

In [207]:
embedding_size = 50

In [4]:
global_document = " ".join([tweet["text"] for tweet in data])
global_document = re.sub(r"\s+", " ", global_document).strip()

In [18]:
global_document = re.sub(r"\s+", " ", global_document).strip()

In [19]:
global_document[:1000]

"dure obama white hous day propos would cost $130 billio An awesom hoodi cat lover print IN usa get i'm founder commun hous / peopl gamer let' connect peopl one spot join server today current make voic count join us dec 6 defend immigr commun cant make tweet call biomed technician 1 rt deathmatch In month decemb call go men woman join hand facin final offici direct telugu titl eleg japanes doll huwari get pussi vibr final usc 31 stanford 28 trojan beat cardin win 2017 pac12 championship rippl xrp price $026 chang 1h 01 market cap $988702997400 rank great game tsk teaser font name effici platform sourc interior product effortlessli beauti jackson walk right thi hondura tonight even realis dirti game talk impact role hous witheunhae171201 mama HK preview big red dildo hello tworld postplay presnuggleroofuzzi moment need chill favourit spot raw vegan pizza delici red pepper flax crust winner announc congratul best vocal perform male solo best ost Wo 1st look star dir releas worldwid jan 1

### Create document for each hashtag

In [22]:
def get_all_hts(tweets):
    all_hts = []
    for tweet in tweets:
        all_hts.extend(tweet["hashtags"])
    return all_hts

In [23]:
all_hts = get_all_hts(data)

In [25]:
all_hts_lowered = list(map(lambda x: x.lower(), all_hts))

In [27]:
all_hts_set = set(all_hts_lowered)

In [28]:
len(all_hts_set)

86660

In [7]:
def create_document_per_ht(tweets):
    hts_to_doc = {}
    for tweet in tweets:
        for hts in tweet["hashtags"]:
            if hts.lower() in hts_to_doc:
                hts_to_doc[hts.lower()] = hts_to_doc[hts.lower()] + " " + tweet["text"]
            else:
                hts_to_doc[hts.lower()] = tweet["text"]
    return hts_to_doc

In [32]:
hts_to_doc = create_document_per_ht(data)

In [8]:
def remove_whitespaces(hts_to_doc):
    hts_to_doc_new = {}
    for key, value in hts_to_doc.items():
        hts_to_doc_new[key] = re.sub(r"\s+", " ", value).strip()
    return hts_to_doc_new

In [36]:
hts_to_doc = remove_whitespaces(hts_to_doc)

### Training word2vec on global document

In [38]:
import gensim

C:\Users\thalv\Anaconda3\envs\dl\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [39]:
sentences = global_document.split(" ")

In [59]:
len(sentences)

1457194

In [243]:
embedding_size

50

In [264]:
model = gensim.models.Word2Vec(min_count=5, size=embedding_size, workers=4, window=5, sg=1)

In [265]:
model.build_vocab([sentences])  # can be a non-repeatable, 1-pass generator

In [78]:
model.corpus_count

1

In [266]:
model.train([sentences], epochs=2000, total_examples=len(sentences)) 

(20000000, 2914388000)

In [80]:
model.wv.vocab

{'dure': <gensim.models.keyedvectors.Vocab at 0x22859f43828>,
 'obama': <gensim.models.keyedvectors.Vocab at 0x22859f43898>,
 'white': <gensim.models.keyedvectors.Vocab at 0x22859f43978>,
 'hous': <gensim.models.keyedvectors.Vocab at 0x22859f43630>,
 'day': <gensim.models.keyedvectors.Vocab at 0x22859f43668>,
 'propos': <gensim.models.keyedvectors.Vocab at 0x22859f43518>,
 'would': <gensim.models.keyedvectors.Vocab at 0x22859f434e0>,
 'cost': <gensim.models.keyedvectors.Vocab at 0x22859f434a8>,
 '$130': <gensim.models.keyedvectors.Vocab at 0x22859f43470>,
 'An': <gensim.models.keyedvectors.Vocab at 0x22859f43438>,
 'awesom': <gensim.models.keyedvectors.Vocab at 0x22859f43400>,
 'hoodi': <gensim.models.keyedvectors.Vocab at 0x22859f433c8>,
 'cat': <gensim.models.keyedvectors.Vocab at 0x22859f43390>,
 'lover': <gensim.models.keyedvectors.Vocab at 0x22859f43358>,
 'print': <gensim.models.keyedvectors.Vocab at 0x22859f43320>,
 'IN': <gensim.models.keyedvectors.Vocab at 0x22859f432e8>,
 'us

In [212]:
model.wv.most_similar(positive="cat", topn=6)

[('0600pm', 0.8374716639518738),
 ('hoodi', 0.8098354935646057),
 ('awesom', 0.7905789613723755),
 ('canberra', 0.7860640287399292),
 ('$130', 0.7814984321594238),
 ('print', 0.7756482362747192)]

In [213]:
model.wv["cat"].shape

(50,)

In [214]:
model.wv[["cat", "awesom"]].shape

(2, 50)

### Create embedding for each hashtag

In [93]:
import numpy as np

In [99]:
"awesom" in model.wv

True

In [9]:
def create_ht_embedding(model, ht_to_doc):
    ht_to_embedding = {}
    for ht, doc in ht_to_doc.items():
        words = doc.split(" ")
        words_in_model = [word for word in words if word in model.wv]
        if words_in_model != []:
            embeddings = model.wv[words_in_model]
            ht_to_embedding[ht] = np.average(embeddings, axis=0)
    return ht_to_embedding

In [267]:
ht_to_embedding = create_ht_embedding(model, hts_to_doc)

In [217]:
ht_to_embedding["prekforall"].shape

(50,)

In [104]:
ht_to_embedding

{'freecommunitycollege': array([  1.23452544e-02,   8.80961493e-02,   6.87067434e-02,
          3.05154566e-02,  -2.66297668e-01,   1.47699723e-02,
          1.60248056e-01,  -1.04858251e-02,   5.45717822e-03,
          1.23452544e-02,   8.80961493e-02,   6.87067434e-02,
          3.05154566e-02,  -2.66297668e-01,   1.47699723e-02,
          1.60248056e-01,  -1.04858251e-02,   5.45717822e-03,
          7.37251539e-05,   1.23452544e-02,   1.10906458e-05,
         -4.07602675e-02,   1.47699723e-02,   1.60248056e-01,
         -1.04858251e-02], dtype=float32),
 'prekforall': array([  1.23452544e-02,   8.80961493e-02,   6.87067434e-02,
          3.05154566e-02,  -2.66297668e-01,   1.47699723e-02,
          1.60248056e-01,  -1.04858251e-02,   5.45717822e-03,
          4.76753246e-03,   1.23452544e-02,   8.80961493e-02,
          6.87067434e-02,   3.05154566e-02,  -2.66297668e-01,
          1.47699723e-02,   1.60248056e-01,  -1.04858251e-02,
          5.45717822e-03,   7.37251539e-05,   1.234

In [129]:
list(model.wv.vocab.keys())

['dure',
 'obama',
 'white',
 'hous',
 'day',
 'propos',
 'would',
 'cost',
 '$130',
 'An',
 'awesom',
 'hoodi',
 'cat',
 'lover',
 'print',
 'IN',
 'usa',
 'get',
 "i'm",
 'founder',
 'commun',
 '/',
 'peopl',
 'gamer',
 "let'",
 'connect',
 'one',
 'spot',
 'join',
 'server',
 'today',
 'current',
 'make',
 'voic',
 'count',
 'us',
 'dec',
 '6',
 'defend',
 'immigr',
 'cant',
 'tweet',
 'call',
 'biomed',
 'technician',
 '1',
 'rt',
 'deathmatch',
 'In',
 'month',
 'decemb',
 'go',
 'men',
 'woman',
 'hand',
 'final',
 'offici',
 'direct',
 'telugu',
 'titl',
 'eleg',
 'japanes',
 'doll',
 'pussi',
 'usc',
 '31',
 'stanford',
 '28',
 'trojan',
 'beat',
 'cardin',
 'win',
 '2017',
 'pac12',
 'championship',
 'rippl',
 'xrp',
 'price',
 'chang',
 '1h',
 '01',
 'market',
 'cap',
 'rank',
 'great',
 'game',
 'tsk',
 'teaser',
 'font',
 'name',
 'effici',
 'platform',
 'sourc',
 'interior',
 'product',
 'effortlessli',
 'beauti',
 'jackson',
 'walk',
 'right',
 'thi',
 'hondura',
 'tonigh

### Testing

In [110]:
import random

In [124]:
random.randint(0, 100)

10

In [125]:
test_tweet = data[random.randint(0, len(data))]

In [10]:
def get_embedding_of_tweet(tweet, model):
    words = tweet["text"].split(" ")
    words_in_model = [word for word in words if word in model.wv]
    if words_in_model != []:
        return np.average(model.wv[words_in_model], axis=0)
    else:
        return np.average(model.wv[list(model.wv.vocab.keys())], axis=0)

In [219]:
test_tweet_embedding = get_embedding_of_tweet(test_tweet, model)

In [220]:
test_tweet_embedding.shape

(50,)

In [135]:
len(data) * .3

64853.399999999994

In [231]:
test_set = np.random.choice(data, size=10000)

In [232]:
len(test_set)

10000

In [141]:
from scipy import spatial

In [11]:
def create_ht_embedding_table(ht_to_embedding):
    '''return table with embeddings and index of ht in table'''
    ht_embedding_table = np.ndarray(shape=(len(ht_to_embedding.keys()), embedding_size), dtype=np.float32)
    i = 0
    idx_to_ht = {}
    for ht, em in ht_to_embedding.items():
        ht_embedding_table[i] = em
        idx_to_ht[i] = ht
        i += 1
    return ht_embedding_table, idx_to_ht

In [268]:
ht_embedding_table, idx_to_ht = create_ht_embedding_table(ht_to_embedding)

In [223]:
ht_embedding_table.shape

(85054, 50)

In [169]:
import scipy

In [12]:
def get_tweets_embedding_table(test_set, model):
    tweet_embedding_table = np.ndarray(shape=(len(test_set), embedding_size), dtype=np.float32)
    i = 0
    n = len(test_set)
    for tweet in test_set:
        print("\r Tweet embeddings: {}/{}".format(i, n), end="")
        tweet_embedding_table[i] = get_embedding_of_tweet(tweet, model)
        i += 1
    return tweet_embedding_table

In [269]:
tweet_embedding_table = get_tweets_embedding_table(test_set, model)

 Tweet embeddings: 9999/10000

In [13]:
def get_closest_ht_to_tweet(tweet, ht_embedding_table, idx_to_ht, model):
    best_sim = 0
    best_ht = ""
    tweet_embedding = get_embedding_of_tweet(tweet, model)
#     for ht, embedding in ht_to_embedding.items():
    cosine_sims = (1 -  scipy.spatial.distance.cdist(ht_embedding_table, tweet_embedding.reshape(1, -1), 'cosine')).reshape(-1)
    max_idx = np.argmax(cosine_sims)
    return idx_to_ht[max_idx]
#         if cosine_sim > best_sim:
#             best_sim = cosine_sim
#             best_ht = ht
#     return best_ht

In [173]:
get_closest_ht_to_tweet(data[123], ht_embedding_table, idx_to_ht, model)

'synecdoche'

In [227]:
ht_embedding_table.shape

(85054, 50)

In [228]:
tweet_embedding_table.shape

(64853, 50)

In [199]:
np.zeros((85054, 10853))

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [117]:
def find_ht_coappearing_with_target(ht_target, tweets):
    ht_list = []
    for tweet in tweets:
        for ht in tweet["hashtags"]:
            if ht.lower() == ht_target.lower():
                ht_list += tweet["hashtags"]
    return set(ht_list)

In [126]:
def test(tweet_embedding_table_k, ht_embedding_table, idx_to_ht, ht_to_idx, tweets, k, model):
    tweet_predictions = []
    counter = 0
    n = len(tweet_embedding_table_k)
#     ht_to_idx = {v: k for k, v in idx_to_ht.items()}

#     tweet_embeddings_table = get_tweets_embedding_table(test_set, model)
    for embed in tweet_embedding_table_k:
        cosine_sim = (1-scipy.spatial.distance.cdist(ht_embedding_table, embed.reshape(1, -1), 'cosine')).reshape(-1)
        max_idx = np.argmax(cosine_sim)
        if k > 1:
            hts = list(find_ht_coappearing_with_target(idx_to_ht[max_idx], tweets))
            hts.sort(key=lambda x: cosine_sim[ht_to_idx[x.lower()]], reverse=True)
            if len(hts) < k:
                topk = hts
            else:  
                topk = hts[:k]
            tweet_predictions.append((counter, topk))
        else:
            tweet_predictions.append((counter, idx_to_ht[max_idx]))
        counter += 1
        print("\r{}/{}".format(counter, n), end="")

#     cosine_sims = (1 -  scipy.spatial.distance.cdist(tweet_embedding_table, ht_embedding_table, 'cosine'))
#     max_idxs = np.argmax(cosine_sims, axis=1)
#     for predicted_idx in max_idxs:
#         print("\r{}/{}".format(counter, n), end="")
#         tweet_predictions.append((test_set[counter], idx_to_ht[predicted_idx]))
#         counter += 1
    return tweet_predictions

In [270]:
test_predictions = test(test_set, ht_embedding_table, idx_to_ht, model)

10000/10000

In [15]:
def f1_aloc(test_set, test_predictions):
    i = 0
    precision = 0
    recall = 0
    for prediction in test_predictions:
        counter , pred_ht  = prediction
        tweet = test_set[counter]
        for ht in tweet["hashtags"]:
            if pred_ht.lower() == ht.lower():
                precision += 1
                recall += 1
                break
    print(precision)
       
    precision /= len(test_set)
    recall /= len(test_set)
    return (2 * precision * recall) / (precision + recall)

In [277]:
f1_aloc(test_set, test_predictions)

2624


0.2624

In [121]:
def f1_muc(test_set, test_predictions, k):
    i = 0
    precision = 0
    recall = 0
    for prediction in test_predictions:
        counter , pred_ht  = prediction
        tweet = test_set[counter]
        prec_current, rec_current = 0, 0
        for ht in tweet["hashtags"]:
            for ht_pr in pred_ht:
                if ht_pr.lower() == ht.lower():
                    prec_current += 1
                    rec_current += 1
        precision += prec_current / k
        recall += rec_current / k
    print(precision)
       
    precision /= len(test_set)
    recall /= len(test_set)
    return (2 * precision * recall) / (precision + recall)

In [123]:
def do_everything(test_set, sentences,hts_to_doc,k,  _model):
    if _model is None:
        model = gensim.models.Word2Vec(min_count=5, size=embedding_size, workers=4, window=5, sg=1)
        model.build_vocab([sentences])  # can be a non-repeatable, 1-pass generator
        print("Training model...")
        model.train([sentences], epochs=2000, total_examples=len(sentences)) 
    else:
        model = _model
        
#         model = gensim.models.Word2Vec(min_count=5, size=embedding_size, workers=4, window=5, sg=1)
#         model.build_vocab([sentences])
#         model.build_vocab([list(pretrained_model.vocab.keys())], update=True)
#         model.intersect_word2vec_format("word2vec_twitter_model.bin",binary=True, lockf=1.0, unicode_errors="ignore")
#         del pretrained_model
#         model.train([sentences], epochs=2000, total_examples=len(sentences))

        
    print("Creating ht embeddings...")
    ht_to_embedding = create_ht_embedding(model, hts_to_doc)
    ht_embedding_table, idx_to_ht = create_ht_embedding_table(ht_to_embedding)
    ht_to_idx = {v: k for k, v in idx_to_ht.items()}

    print("Creating test tweet embeddings...")
    tweet_embedding_table = get_tweets_embedding_table(test_set, model)
    print("")
    print("Predicting...")
    test_predictions = test(tweet_embedding_table, ht_embedding_table, idx_to_ht, ht_to_idx, test_set, k, model)
    print(" ")
    if k == 1:
        print("F1: {}".format(f1_aloc(test_set, test_predictions)))
    else:
        print("F1 at k={}: {}".format(k, f1_muc(test_set, test_predictions, k)))



In [308]:
test_set_20k = np.random.choice(data, size=20000)

In [309]:
do_everything(test_set_20k, sentences)

Training model...
Creating ht embeddings...
Creating test tweet embeddings...
 Tweet embeddings: 19999/20000
Predicting...
20000/20000 
5235
F1: 0.26175


In [285]:
with open("all_preproc_no_ht_symbols.pickle", 'rb') as f:
    data_with_ht = pickle.load(f)

In [286]:
global_document_ht = " ".join([tweet["text"] for tweet in data_with_ht])
global_document_ht = re.sub(r"\s+", " ", global_document_ht).strip()
hts_to_doc_ht = create_document_per_ht(data_with_ht)
hts_to_doc_ht = remove_whitespaces(hts_to_doc_ht)
sentences_ht = global_document_ht.split(" ")
test_set_ht = np.random.choice(data_with_ht, size=1000)

In [303]:
test_set_ht = np.random.choice(data_with_ht, size=10000)

In [306]:
do_everything(test_set_ht, sentences_ht)

Training model...
Creating ht embeddings...
Creating test tweet embeddings...
 Tweet embeddings: 9999/10000
Predicting...
10000/10000 
2039
F1: 0.2039


### Pretrained

In [5]:
with open("no_stem_no_hashtags.pickle", 'rb') as f:
    data_no_ht = pickle.load(f)

In [26]:
def count_hts(tweets):
    counter = {}
    for tweet in tweets:
        for ht in tweet["hashtags"]:
            if ht.lower() in counter:
                counter[ht.lower()] = counter[ht.lower()] + 1
            else:
                counter[ht.lower()] = 1
    return counter

In [28]:
def filter_only_common_hts(tweets):
    counter = count_hts(tweets)
    hts = set()
    for ht, count in counter.items():
        if count >= 200 and count <= 500:
            hts.add(ht)
    return hts

In [30]:
common_hts = filter_only_common_hts(data_no_ht)

In [84]:
def create_document_per_ht(tweets):
    hts_to_doc = {}
    for tweet in tweets:
        for hts in tweet["hashtags"]:
#             if hts.lower():
#                 continue
            if hts.lower() in hts_to_doc:
                hts_to_doc[hts.lower()] = hts_to_doc[hts.lower()] + " " + tweet["text"]
            else:
                hts_to_doc[hts.lower()] = tweet["text"]
    return hts_to_doc

In [85]:
hts_to_doc = create_document_per_ht(data_no_ht)

In [86]:
len(hts_to_doc)

86666

In [3]:
import gensim

C:\Users\thalv\Anaconda3\envs\dl\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [104]:
model_twi = gensim.models.KeyedVectors.load_word2vec_format('word2vec_twitter_model.bin', binary=True, unicode_errors="ignore")  

In [19]:
import numpy as np

In [87]:
embedding_size = 300

In [88]:
global_document_ht = " ".join([tweet["text"] for tweet in data_no_ht])
global_document_ht = re.sub(r"\s+", " ", global_document_ht).strip()
sentences_ht = global_document_ht.split(" ")
test_set_ht = np.random.choice(data_no_ht, size=1000)
common_hts = filter_only_common_hts(data_no_ht)
# hts_to_doc = create_document_per_ht(data_no_ht, common_hts)
hts_to_doc_ht = create_document_per_ht(data_no_ht)
hts_to_doc_ht = remove_whitespaces(hts_to_doc_ht)

In [58]:
all_words = global_document_ht.split(" ")
words_in_dict = []
for word in all_words:
    if word in model_gogl.wv:
        words_in_dict.append(word)
        
subset = model_gogl.wv[words_in_dict]

C:\Users\thalv\Anaconda3\envs\dl\lib\site-packages\ipykernel\__main__.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\thalv\Anaconda3\envs\dl\lib\site-packages\ipykernel\__main__.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [92]:
all_words = global_document_ht.split(" ")
words_in_dict = []
for word in all_words:
    if word in model_twi.wv:
        words_in_dict.append(word)
subset_twi = model_twi.wv[words_in_dict]

C:\Users\thalv\Anaconda3\envs\dl\lib\site-packages\ipykernel\__main__.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\thalv\Anaconda3\envs\dl\lib\site-packages\ipykernel\__main__.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


#### dummy model

In [70]:
class Dummmmmy():
    def __init__(self, keys):
        self.keys = keys
    def keys(self):
        return self.keys
    
class DictMultipleValues(dict):
    def __init__(self):
        dict.__init__(self)
        self.vocab = Dummmmmy(self.keys)
        
    def __getitem__(self, keys):
        if type(keys) == type([]):
            return [dict.__getitem__(self, key) for key in keys]
        else:
            return dict.__getitem__(self, keys)
        

In [71]:
d = DictMultipleValues()
d[1]=2
d[2]=3
print(d.vocab.keys())

dict_keys([1, 2])


In [72]:
class DummyModel():
    def __init__(self, words_in_model, embeddings):
        self.wv = DictMultipleValues()
        counter = 0
        for word in words_in_dict:
            self.wv[word] = embeddings[counter]
            counter += 1

In [73]:
dummy_model = DummyModel(words_in_dict, subset)

In [98]:
dummy_twitter = DummyModel(words_in_dict, subset_twi)
embedding_sizeding_size = 400

In [74]:
len(dummy_model.wv)

42834

In [95]:
# del words_in_dict
# del subset
del model_twi

In [78]:
test_set_ht = np.random.choice(data_no_ht, size=10000)

In [96]:
len(test_set_ht)

1000

In [76]:
import scipy

In [106]:
do_everything(test_set_ht, sentences_ht, hts_to_doc, model_twi)

Creating ht embeddings...
Creating test tweet embeddings...
 Tweet embeddings: 999/1000
Predicting...
1000/1000 
364
F1: 0.364


In [99]:
do_everything(test_set_ht, sentences_ht, hts_to_doc, dummy_twitter)

Creating ht embeddings...
Creating test tweet embeddings...
 Tweet embeddings: 999/1000
Predicting...
1000/1000 
367
F1: 0.367


In [89]:
do_everything(test_set_ht, sentences_ht, hts_to_doc, dummy_twitter)

Creating ht embeddings...
Creating test tweet embeddings...
 Tweet embeddings: 999/1000
Predicting...
1000/1000 
313
F1: 0.313


In [108]:
with open("no_stem_no_hashtag_symbols.pickle", 'rb') as f:
    data_no_sym = pickle.load(f)

In [109]:
global_document_sym = " ".join([tweet["text"] for tweet in data_no_sym])
global_document_sym = re.sub(r"\s+", " ", global_document_sym).strip()
sentences_sym = global_document_sym.split(" ")
test_set_sym = np.random.choice(data_no_sym, size=1000)
# common_hts = filter_only_common_hts(data_no_ht)
# hts_to_doc = create_document_per_ht(data_no_ht, common_hts)
hts_to_doc_sym = create_document_per_ht(data_no_sym)
hts_to_doc_sym = remove_whitespaces(hts_to_doc_sym)

In [110]:
model_twi = gensim.models.KeyedVectors.load_word2vec_format('word2vec_twitter_model.bin', binary=True, unicode_errors="ignore")  

In [111]:
all_words_sym = global_document_sym.split(" ")
words_in_dict_sym = []
for word in all_words_sym:
    if word in model_twi.wv:
        words_in_dict_sym.append(word)
subset_twi_sym = model_twi.wv[words_in_dict_sym]

C:\Users\thalv\Anaconda3\envs\dl\lib\site-packages\ipykernel\__main__.py:4: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\thalv\Anaconda3\envs\dl\lib\site-packages\ipykernel\__main__.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [112]:
dummy_twitter = DummyModel(words_in_dict_sym, subset_twi_sym)

In [113]:
del model_twi

In [114]:
do_everything(test_set_sym, sentences_sym, hts_to_doc_sym, dummy_twitter)

Creating ht embeddings...
Creating test tweet embeddings...
 Tweet embeddings: 999/1000
Predicting...
1000/1000 
483
F1: 0.483


In [127]:
do_everything(test_set_sym, sentences_sym, hts_to_doc_sym, 3, dummy_twitter)

Creating ht embeddings...
Creating test tweet embeddings...
 Tweet embeddings: 999/1000
Predicting...
1000/1000 
427.0000000000004
F1 at k=3: 0.4270000000000004
